# Deploying a model in AWS Sagemaker

In [22]:
from sagemaker.sklearn.estimator import SKLearn

role = "arn:aws:iam::058264225877:role/service-role/AmazonSageMaker-ExecutionRole-20240414T183938"
testPath = "s3://cisc498-data/credit-card-fraud/test/fraudTest.csv"
trainPath = "s3://cisc498-data/credit-card-fraud/train/fraudTrain.csv"

# Initialize a SageMaker SKLearn estimator
sklearn_estimator = SKLearn(
    entry_point='train.py', # Script to train the model
    framework_version='0.23-1',
    role=role,
    instance_type='ml.m5.large',
    instance_count=1,
    hyperparameters={'n_estimators': 300, 'random_state': 42}
)

In [23]:
sklearn_estimator.fit({"train": trainPath, "test": testPath}, wait=True)

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-05-17-03-06-31-686


2024-05-17 03:06:31 Starting - Starting the training job...
2024-05-17 03:06:47 Starting - Preparing the instances for training...
2024-05-17 03:07:19 Downloading - Downloading input data...
2024-05-17 03:07:53 Downloading - Downloading the training image...
2024-05-17 03:08:29 Training - Training image download completed. Training in progress..2024-05-17 03:08:34,046 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-05-17 03:08:34,049 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-17 03:08:34,089 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-05-17 03:08:34,245 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-17 03:08:34,257 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-17 03:08:34,270 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-17 03:

In [24]:
import boto3
import sagemaker

# Initialize SageMaker session
sagemaker_session = sagemaker.Session()

# Initialize SageMaker client
sm_boto3 = boto3.client('sagemaker')

sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]
print(artifact)


2024-05-17 03:10:22 Starting - Preparing the instances for training
2024-05-17 03:10:22 Downloading - Downloading the training image
2024-05-17 03:10:22 Training - Training image download completed. Training in progress.
2024-05-17 03:10:22 Uploading - Uploading generated training model
2024-05-17 03:10:22 Completed - Training job completed
s3://sagemaker-us-east-1-058264225877/sagemaker-scikit-learn-2024-05-17-03-06-31-686/output/model.tar.gz


In [25]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "fraud-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name = model_name,
    model_data=artifact,
    role="arn:aws:iam::058264225877:role/service-role/AmazonSageMaker-ExecutionRole-20240414T183938",
    entry_point="train.py",
    framework_version='0.23-1',
)

In [26]:
predictor = model.deploy(initial_instance_count=1, instance_type="ml.m5.large", endpoint_name=model_name,)

INFO:sagemaker:Creating model with name: fraud-model-2024-05-17-03-13-58
INFO:sagemaker:Creating endpoint-config with name fraud-model-2024-05-17-03-13-58
INFO:sagemaker:Creating endpoint with name fraud-model-2024-05-17-03-13-58


-----!

## Inferring model

In [27]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

endpoint_name = "fraud-model-2024-05-17-03-13-58"
content_type = 'application/json'
payload = json.dumps([[219.66, 41.14, -84.5722, 6284, 41.853691, -84.255284]])
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload
    )

print(response['Body'].read().decode())

[0]
